## 베이즈 정리

검사 시약 문제: 제약사에서 환자가 특정한 병에 걸렸는지 확인하는 시약을 만듦. 
환자에게 시약을 테스트한 결과 99%의 정확도를 나타 냄.

이 때, 일반적인 사람의 시약 테스트가 양성이 나타난 경우 실제 이 병에 걸렸을 확률?

- 이 병은 전체 인구 중 걸린 사람이 `0.2%`
- 이 병에 걸리지 않은 사람의 시약 테스트가 양성으로 나온 확률은`5%`

In [7]:
from pgmpy.factors.discrete import TabularCPD

cpd_X = TabularCPD('X', 2, [[0.998], [0.002]])
print(cpd_X)

+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


In [8]:
import numpy as np
import pandas as pd
#시각화 패키지
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

if platform.system() == 'Darwin': rc('font', family='AppleGothic')

import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False

In [10]:
# 진단 시약의 판단 여부를 대입해서 확률 모형을 생성
# 정상인 경우 정상으로 판단 95% 잘못 판단 5%
# 병에 걸린 경우 병에 걸리지 않았다고 판단 1%, 양성으로 판단 99%

# (X(0), Y(0)): 정상인 경우 정상으로 판단.
# (X(0), Y(1)): 정상인 경우 잘못 판단.
# (X(1), Y(0)): 병에 걸린 경우 잘못 판단.
# (X(1), Y(1)): 병에 걸린 경우 양성으로 판단.

cpd_Y_on_X = TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]),
                        evidence=['X'], evidence_card=[2])

print(cpd_Y_on_X)

+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


/var/folders/8y/qly304_55c1cf35dff76v_lh0000gn/T/ipykernel_17943/1185197337.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  cpd_Y_on_X = TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]),


In [14]:
from pgmpy.models import BayesianNetwork

model = BayesianNetwork([('X', 'Y')])
model.add_cpds(cpd_X, cpd_Y_on_X)

from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

# Y값이 1이면 양성으로 나왔을 때 X(0): 병에 걸리지 않았을 확률, X(1): 병에 걸릴 확률
# Y값이 0이면 음성으로 나왔을 때 X(0): 병에 걸리지 않았을 확률, X(1): 병에 걸릴 확률
print(inference.query(['X'], evidence={'Y': 1}))

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+
